In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import sys
import cv2 as cv
from tqdm import tqdm

from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u

In [2]:
combine_fold = 'C:/Users/oryan/Documents/mergers-in-cosmos/cats-to-combine'
data_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/back-to-basics-data'
fig_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/paper-source/figures'
cos_dat_fold = 'E:/cosmos-data'

In [3]:
df_cat = pd.read_csv(f'{combine_fold}/cosmos2020-categorised-interactions.csv', index_col = 0)

In [4]:
df_cat

,ORyan23_ID,ID,category,ALPHA_J2000,DELTA_J2000,X_IMAGE,Y_IMAGE,ERRX2_IMAGE,ERRY2_IMAGE,ERRXY_IMAGE,...,ez_ssfr_p025,ez_ssfr_p160,ez_ssfr_p500,ez_ssfr_p840,ez_ssfr_p975,ez_Av_p025,ez_Av_p160,ez_Av_p500,ez_Av_p840,ez_Av_p975
0,4000705532455,816891,stage3,150.679846,2.196543,9200.399414,21737.232422,0.000052,0.000059,-1.043282e-05,...,-8.931825,-8.911355,-8.899668,-8.883307,-8.872990,0.529107,0.560142,0.599624,0.630920,0.652330
1,4000705533383,861738,stage1,150.645118,2.237538,10033.689453,22720.843750,0.000039,0.000078,1.332813e-05,...,-9.574272,-9.548649,-9.503962,-9.439343,-9.384205,0.240042,0.281559,0.330791,0.384371,0.436491
2,4000705539529,1284864,stage1,149.686223,2.637412,33027.406250,32317.517578,0.000031,0.000030,1.244353e-06,...,-9.669377,-9.614943,-9.570104,-9.533121,-9.476167,0.094187,0.114436,0.140990,0.159430,0.183608
3,4000705539941,1301188,stage2,149.702462,2.654895,32637.949219,32737.013672,0.000064,0.000104,-2.881107e-06,...,-9.794128,-9.784252,-9.766013,-9.743796,-9.727879,0.018737,0.018737,0.018737,0.018737,0.018737
4,4000705540034,1304538,stage1,149.726828,2.658172,32053.712891,32815.468750,0.000071,0.000086,-6.036952e-06,...,-9.480532,-9.443615,-9.402433,-9.352081,-9.325659,0.302629,0.376027,0.440394,0.508153,0.567727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,6000535066150,1157587,stage4,150.422510,2.514655,15373.875977,29370.304688,0.000024,0.000027,8.697323e-06,...,-13.220035,-13.215519,-10.469687,-9.954029,-9.691111,0.173505,0.240862,0.328586,0.439079,0.546386
3792,6000535121452,1035987,stage1,149.694761,2.397231,32824.328125,26552.759766,0.000012,0.000015,-7.174042e-07,...,-10.287872,-10.060802,-9.853634,-9.456022,-9.315375,0.343488,0.463897,0.646257,1.016819,1.285051
3793,6000535865556,1138628,stage1,150.490889,2.496660,13734.171875,28938.828125,0.001929,0.001333,1.953999e-04,...,-9.875473,-9.817194,-9.580817,-9.265499,-9.099182,0.026873,0.026873,0.164250,0.549086,0.751945
3794,6000536013605,1203157,stage1,150.202813,2.558504,20641.730469,30421.910156,0.000051,0.000067,4.840965e-07,...,-9.804204,-9.683063,-9.470870,-9.189605,-8.982660,0.539500,0.595082,0.778112,1.062422,1.232495


In [5]:
df_red_tmp = df_cat.query('ez_sfr < 10')
df_red = df_red_tmp.query('lp_mass_best > 6.5')

### Cross Matching with AGN Catalogues

In [6]:
df_red

,ORyan23_ID,ID,category,ALPHA_J2000,DELTA_J2000,X_IMAGE,Y_IMAGE,ERRX2_IMAGE,ERRY2_IMAGE,ERRXY_IMAGE,...,ez_ssfr_p025,ez_ssfr_p160,ez_ssfr_p500,ez_ssfr_p840,ez_ssfr_p975,ez_Av_p025,ez_Av_p160,ez_Av_p500,ez_Av_p840,ez_Av_p975
0,4000705532455,816891,stage3,150.679846,2.196543,9200.399414,21737.232422,0.000052,0.000059,-1.043282e-05,...,-8.931825,-8.911355,-8.899668,-8.883307,-8.872990,0.529107,0.560142,0.599624,0.630920,0.652330
1,4000705533383,861738,stage1,150.645118,2.237538,10033.689453,22720.843750,0.000039,0.000078,1.332813e-05,...,-9.574272,-9.548649,-9.503962,-9.439343,-9.384205,0.240042,0.281559,0.330791,0.384371,0.436491
2,4000705539529,1284864,stage1,149.686223,2.637412,33027.406250,32317.517578,0.000031,0.000030,1.244353e-06,...,-9.669377,-9.614943,-9.570104,-9.533121,-9.476167,0.094187,0.114436,0.140990,0.159430,0.183608
3,4000705539941,1301188,stage2,149.702462,2.654895,32637.949219,32737.013672,0.000064,0.000104,-2.881107e-06,...,-9.794128,-9.784252,-9.766013,-9.743796,-9.727879,0.018737,0.018737,0.018737,0.018737,0.018737
4,4000705540034,1304538,stage1,149.726828,2.658172,32053.712891,32815.468750,0.000071,0.000086,-6.036952e-06,...,-9.480532,-9.443615,-9.402433,-9.352081,-9.325659,0.302629,0.376027,0.440394,0.508153,0.567727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,6000535066150,1157587,stage4,150.422510,2.514655,15373.875977,29370.304688,0.000024,0.000027,8.697323e-06,...,-13.220035,-13.215519,-10.469687,-9.954029,-9.691111,0.173505,0.240862,0.328586,0.439079,0.546386
3792,6000535121452,1035987,stage1,149.694761,2.397231,32824.328125,26552.759766,0.000012,0.000015,-7.174042e-07,...,-10.287872,-10.060802,-9.853634,-9.456022,-9.315375,0.343488,0.463897,0.646257,1.016819,1.285051
3793,6000535865556,1138628,stage1,150.490889,2.496660,13734.171875,28938.828125,0.001929,0.001333,1.953999e-04,...,-9.875473,-9.817194,-9.580817,-9.265499,-9.099182,0.026873,0.026873,0.164250,0.549086,0.751945
3794,6000536013605,1203157,stage1,150.202813,2.558504,20641.730469,30421.910156,0.000051,0.000067,4.840965e-07,...,-9.804204,-9.683063,-9.470870,-9.189605,-8.982660,0.539500,0.595082,0.778112,1.062422,1.232495


In [7]:
df_coords = (
    df_red[['ID', 'ALPHA_J2000', 'DELTA_J2000']]
    .rename(columns = {'ALPHA_J2000':'RA','DELTA_J2000':'DEC'})
    .set_index('ID')
)

In [8]:
coords_dict = df_coords.to_dict(orient='index')

### Smolcic Catalogue 

In [9]:
with fits.open(f'{cos_dat_fold}/VLA_3GHz_counterpart_array_20170210_paper_smolcic_et_al.fits') as hdul:
    smol_rec = hdul[1].data

In [10]:
smol_rec.names

['ID_VLA',
 'RA_VLA_J2000',
 'DEC_VLA_J2000',
 'MULTI',
 'CAT_CPT',
 'ID_CPT',
 'RA_CPT_J2000',
 'DEC_CPT_J2000',
 'SEP_VLA_CPT',
 'P_FALSE',
 'Z_BEST',
 'FLUX_INT_3GHz',
 'Lradio_10cm',
 'Lradio_21cm',
 'L_TIR_SF',
 'SFR_KENN98',
 'Xray_AGN',
 'MIR_AGN',
 'SED_AGN',
 'Quiescent_MLAGN',
 'SFG',
 'Clean_SFG',
 'HLAGN',
 'MLAGN',
 'Radio_excess',
 'flag_COSMOS2015']

In [11]:
# Function that selects the closest object in the MPA_JHU as the galaxy of interest.
def get_correct_smolcic_id(ra, dec, tab):
    cosmos_coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    mpa_coords = SkyCoord(ra = tab['RA_VLA_J2000'] * u.deg, dec = tab['DEC_VLA_J2000'] * u.deg, frame = 'fk5')
    
    seps = list(abs((cosmos_coord.separation(mpa_coords)).to(u.arcsec) / u.arcsec))
    
    min_ind = seps.index(min(seps))
    
    return {'smol_ID' : tab[min_ind]['ID_VLA'], 'RA' : tab[min_ind]['RA_VLA_J2000'], 'Dec' : tab[min_ind]['DEC_VLA_J2000']}

In [12]:
def matching_smolcic_ids(ra, dec, smol_rec):
    arcsec_sep_deg = 30 / (60 * 60)
    
    matching_tab = smol_rec[(smol_rec['RA_VLA_J2000'] > ra - arcsec_sep_deg) & (smol_rec['RA_VLA_J2000'] < ra + arcsec_sep_deg) & (smol_rec['DEC_VLA_J2000'] > dec - arcsec_sep_deg) & (smol_rec['DEC_VLA_J2000'] < dec + arcsec_sep_deg)]
    if len(matching_tab) < 0.5:
        return {'smol_ID' : np.nan, 'RA' : np.nan, 'Dec' : np.nan}
    elif len(matching_tab) == 1:
        return {'smol_ID' : matching_tab['ID_VLA'][0], 'RA' : matching_tab['RA_VLA_J2000'][0], 'Dec' : matching_tab['DEC_VLA_J2000'][0]}
    elif len(matching_tab) > 1:
        return get_correct_smolcic_id(ra, dec, matching_tab)
    else:
        print('Somethings gone very wrong!!')
        sys.exit()

In [13]:
smolcic_matched_dict = {}

In [14]:
done_ids = list(smolcic_matched_dict.keys())
for i in tqdm(list(coords_dict.keys())):
    if i in done_ids:
        continue
    
    smolcic_matched_dict[i] = matching_smolcic_ids(coords_dict[i]['RA'], coords_dict[i]['DEC'], smol_rec)

100%|██████████| 3772/3772 [00:54<00:00, 69.34it/s]


In [15]:
smolcic_matched_df = pd.DataFrame.from_dict(smolcic_matched_dict, orient = 'index').dropna().reset_index().rename(columns = {'index' : 'ID', 0 : 'smol_id'})

In [16]:
def dedup_smol(df):
    
    ra = df.RA.iloc[0]
    dec = df.Dec.iloc[0]
    
    if len(df) == 1:
        return df.ID.iloc[0]
    
    coords = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    df_coords = df.assign(coords = df.apply(lambda row: SkyCoord(ra = row.ALPHA_J2000 * u.deg, dec = row.DELTA_J2000 * u.deg, frame = 'fk5'), axis = 1))
    
    df_seps = df_coords.assign(seps = df_coords.coords.apply(lambda x: np.float64(abs(((x.separation(coords)).to(u.arcsec) / u.arcsec)))))
    min_sep = df_seps.seps.min()
    min_row = df_seps.query('seps == @min_sep')
    return int(min_row.ID.iloc[0])

In [17]:
df_smol_merg = (
    smolcic_matched_df
    .merge(df_cat, on = 'ID', how = 'left')
    [['ID', 'smol_ID', 'RA', 'Dec', 'ALPHA_J2000', 'DELTA_J2000']]
)

In [18]:
smol_dedup_df = pd.DataFrame(df_smol_merg.groupby('smol_ID').apply(lambda df: dedup_smol(df))).reset_index().rename(columns = {0 : 'ID'})

In [19]:
smol_dedup_df

,smol_ID,ID
0,1.0,702861
1,3.0,1222315
2,5.0,541286
3,8.0,888234
4,17.0,191110
...,...,...
2060,10919.0,517235
2061,10928.0,1443968
2062,10940.0,843319
2063,10945.0,1157455


In [20]:
def getting_smol_info(smol_id, smol_rec):
    row = smol_rec[smol_rec['ID_VLA'] == smol_id]
    clsfs = [row['SFG'][0], row['Clean_SFG'][0]]
    
    if clsfs[0] == 'true' and clsfs[1] == 'true':
        return 'SFG'
    elif clsfs[0] == 'false' and clsfs[1] == 'false':
        return 'AGN'
    elif clsfs[0] == 'true' and clsfs[1] == 'false':
        return 'SFG'
    elif clsfs[0] == 'false' and clsfs[1] == 'true':
        print('Something wrong: 1')
        sys.exit()

In [21]:
smol_clsf_dict = {}

In [22]:
smol_dedup_dict = smol_dedup_df.set_index('ID').to_dict(orient = 'index')

In [23]:
done_ids = list(smol_clsf_dict.keys())
for i in tqdm(list(smol_dedup_dict.keys())):
    if i in done_ids:
        continue
    
    smol_clsf_dict[i] = getting_smol_info(smol_dedup_dict[i]['smol_ID'], smol_rec)

100%|██████████| 2065/2065 [00:20<00:00, 101.98it/s]


In [24]:
smol_clsf_df = pd.DataFrame.from_dict(smol_clsf_dict, orient = 'index').reset_index().rename(columns = {'index' : 'ID', 0 : 'agn_clsf'})

In [25]:
smol_clsf_df.agn_clsf.value_counts()

SFG    1339
AGN     726
Name: agn_clsf, dtype: int64

In [26]:
smol_cat_ids = list(smol_clsf_df.ID)

### The MPA-JHU

In [28]:
with fits.open(f'{cos_dat_fold}/MPA-JHU_full_catalogue.fits') as hdul:
    mpa_jhu = hdul[1].data

In [29]:
matched_dict = {}

In [30]:
# Function that selects the closest object in the MPA_JHU as the galaxy of interest.
def get_correct_id(ra, dec, tab):
    cosmos_coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    mpa_coords = SkyCoord(ra = tab['RA'] * u.deg, dec = tab['DEC'] * u.deg, frame = 'fk5')
    
    seps = list(abs((cosmos_coord.separation(mpa_coords)).to(u.arcsec) / u.arcsec))
    
    min_ind = seps.index(min(seps))
    
    return {'SpecID' : tab[min_ind]['SPECOBJID_1'], 'RA' : tab[min_ind]['RA'], 'Dec' : tab[min_ind]['DEC']}

In [31]:
def matching_ids(ra, dec, mpa_jhu):
    arcsec_sep_deg = 30 / (60 * 60)
    
    matching_tab = mpa_jhu[(mpa_jhu['RA'] > ra - arcsec_sep_deg) & (mpa_jhu['RA'] < ra + arcsec_sep_deg) & (mpa_jhu['DEC'] > dec - arcsec_sep_deg) & (mpa_jhu['DEC'] < dec + arcsec_sep_deg)]
    if len(matching_tab) < 0.5:
        return {'SpecID' : np.nan, 'RA' : np.nan, 'Dec' : np.nan}
    elif len(matching_tab) == 1:
        return {'SpecID' : matching_tab['SPECOBJID_1'][0], 'RA' : matching_tab['RA'][0], 'Dec' : matching_tab['DEC'][0]}
    elif len(matching_tab) > 1:
        return get_correct_id(ra, dec, matching_tab)
    else:
        print('Somethings gone very wrong!!')
        sys.exit()

In [32]:
done_ids = list(matched_dict.keys())
for i in tqdm(list(coords_dict.keys())):
    if i in done_ids:
        continue
    
    matched_dict[i] = matching_ids(coords_dict[i]['RA'], coords_dict[i]['DEC'], mpa_jhu)

100%|██████████| 3772/3772 [24:38<00:00,  2.55it/s]


In [33]:
df_mpa_matched_lists = pd.DataFrame.from_dict(matched_dict, orient = 'index').rename(columns = {0: 'SpecID_List'})

In [36]:
df_mpa_matched = (
    df_mpa_matched_lists
    .dropna()
    .reset_index()
    .rename(columns = {'index' : 'ID'})
)

In [37]:
mpa_checked_df = (
    df_mpa_matched
    .assign(in_mpa = df_mpa_matched.ID.apply(lambda x: 'y' if x in smol_cat_ids else 'n'))
)

In [38]:
mpa_rem = (
    mpa_checked_df.query('in_mpa == "n"').drop(columns = 'in_mpa')
)

In [39]:
def dedup_mpa(df):
    
    
    ra = df.RA.iloc[0]
    dec = df.Dec.iloc[0]
    
    if len(df) == 1:
        return df.ID.iloc[0]
    
    coords = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    df_coords = df.assign(coords = df.apply(lambda row: SkyCoord(ra = row.ALPHA_J2000 * u.deg, dec = row.DELTA_J2000 * u.deg, frame = 'fk5'), axis = 1))
    
    df_seps = df_coords.assign(seps = df_coords.coords.apply(lambda x: np.float64(abs(((x.separation(coords)).to(u.arcsec) / u.arcsec)))))
    min_sep = df_seps.seps.min()
    min_row = df_seps.query('seps == @min_sep')
    return min_row.ID.iloc[0]

In [40]:
list(df_mpa_matched.columns)

['ID', 'SpecID', 'RA', 'Dec']

In [41]:
df_mpa_merg = (
    df_mpa_matched
    .merge(df_cat, on = 'ID', how = 'left')
    [['ID', 'SpecID', 'RA', 'Dec', 'ALPHA_J2000', 'DELTA_J2000']]
)

In [42]:
df_dedup = pd.DataFrame(df_mpa_merg.groupby('SpecID').apply(lambda df: dedup_mpa(df))).reset_index().rename(columns = {0 : 'ID'})

In [43]:
df_dedup

,SpecID,ID
0,541731884917352448,574669
1,562955759311153152,287714
2,562969228328593408,233386
3,562970327840221184,331101
4,562981048078592000,293597
...,...,...
88,564194638081058816,1184170
89,564194912958965760,828217
90,564195187836872704,949863
91,564195737592686592,1074035


In [46]:
df_spec_dict = df_dedup.set_index('ID').to_dict(orient = 'index')

In [48]:
spec_details = {}

In [49]:
def getting_info(specid, mpa_jhu):
    row = mpa_jhu[mpa_jhu['SPECOBJID_1'] == str(specid)]
    columns = row.names
    detail_dict = {}
    for j in columns:
        detail_dict[j] = row[j][0]
    
    return detail_dict

In [50]:
done_ids = list(spec_details.keys())
for i in tqdm(list(df_spec_dict.keys())):
    if i in done_ids:
        continue
    
    spec_details[i] = getting_info(df_spec_dict[i]['SpecID'], mpa_jhu)

100%|██████████| 93/93 [00:52<00:00,  1.76it/s]


In [51]:
spec_details_df = pd.DataFrame.from_dict(spec_details, orient = 'index')

In [52]:
spec_details_df

,SPECOBJID_1,PLATEID_1,MJD_1,FIBERID_1,BPTCLASS,LGM_TOT_P2P5,LGM_TOT_P16,LGM_TOT_P50,LGM_TOT_P84,LGM_TOT_P97P5,...,OII_FLUX,OII_FLUX_ERR,OII_VOFF,OII_CHI2,OIII_SIGMA,OIII_FLUX,OIII_FLUX_ERR,OIII_VOFF,OIII_CHI2,SPECTOFIBER
574669,541731884917352448,481,51908,633,2,11.078977,11.166218,11.271535,11.372645,11.467699,...,20.553787,51.536793,-74.665283,1.494422,1.000000,4.364240,1.484928,130.245636,1.692840,0.821275
287714,562955759311153152,500,51994,21,-1,11.139956,11.219204,11.310024,11.395944,11.488194,...,-3.216977,3.366987,120.474434,1.402987,79.940910,3.064303,2.149659,63.333294,0.236586,0.918593
233386,562969228328593408,500,51994,70,2,10.015815,10.084843,10.173661,10.268553,10.361866,...,38.899212,6.768869,-31.992714,0.346235,39.245960,4.414524,2.144435,4.255847,0.507827,0.844826
331101,562970327840221184,500,51994,74,-1,5.856493,6.024266,6.130666,6.288399,6.598499,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.880991
293597,562981048078592000,500,51994,113,3,10.613132,10.695285,10.793767,10.894615,10.995457,...,38.967434,176.637741,-49.209496,0.894294,193.300613,22.884871,4.797235,-48.351486,0.475114,0.883931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184170,564194638081058816,501,52235,432,2,10.800610,10.903141,11.009504,11.116230,11.226132,...,12.859809,2.410445,-4.343460,0.394762,62.264637,6.672960,1.690545,93.381584,0.199612,0.773150
828217,564194912958965760,501,52235,433,-1,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,...,0.000000,1.063794,17.733913,0.000000,371.071411,10.565377,5.689797,300.000000,0.753438,0.752216
949863,564195187836872704,501,52235,434,-1,11.297916,11.386047,11.482634,11.574331,11.677234,...,7.056612,38.293255,116.003349,0.551523,1.000000,2.118363,0.864428,-155.439499,0.992333,0.748879
1074035,564195737592686592,501,52235,436,-1,11.074389,11.144280,11.224672,11.309345,11.391687,...,29.905798,243.225861,-104.943184,1.125899,96.275513,5.049606,2.232223,-121.546783,0.893146,0.720676


In [53]:
spec_details_df.BPTCLASS.value_counts()

-1    50
 2    15
 1    14
 3     7
 4     4
 5     3
Name: BPTCLASS, dtype: int64

Note for above: The categories are -1 = unclassifiable, 0 = Not used, 1 = Star Forming, 2 - Low S/N Star Forming, 3 = Composite, 4 = AGN Non-Liner, 5 = Low S/N Liner

In [54]:
spec_classification = spec_details_df.query('BPTCLASS >= 1').reset_index().rename(columns = {'index' : 'ID'})[['ID', 'BPTCLASS']]

In [55]:
def classify_ID(clsf):
    if clsf == 1:
        return 'SF'
    elif clsf == 2:
        return 'SF'
    elif clsf == 3:
        return 'Composite'
    elif clsf == 4:
        return 'AGN_Non_Liner'
    elif clsf == 5:
        return 'AGN_Liner'
    else:
        print('What')
        sys.exit()

In [56]:
clsf_df = (
    spec_classification
    .assign(agn_clsf = spec_classification.BPTCLASS.apply(lambda x: classify_ID(x)))
    .drop(columns = 'BPTCLASS')
)

In [57]:
del mpa_jhu

### The VLA 3GHz Counterpart Array

In [58]:
with fits.open(f'{cos_dat_fold}/VLA_3GHz_counterpart_array_20170210_paper_delvecchio_et_al.fits') as hdul:
    vla_cat = hdul[1].data

In [59]:
vla_cat.names

['ID_VLA3',
 'RA_VLA3',
 'DEC_VLA3',
 'Z_BEST',
 'Z_TYPE',
 'FLUX_INT_3GHz',
 'Lradio_10cm',
 'Lradio_21cm',
 'L_TIR_SF',
 'flag_herschel3',
 'Mstar',
 'SFR_IR',
 'NUV_R',
 'XRAY_AGN',
 'MIR_AGN',
 'SED_AGN',
 'Radio_excess',
 'Class']

In [60]:
len(vla_cat)

7729

In [61]:
# Function that selects the closest object in the MPA_JHU as the galaxy of interest.
def get_correct_vla_id(ra, dec, tab):
    cosmos_coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    mpa_coords = SkyCoord(ra = tab['RA_VLA3'] * u.deg, dec = tab['DEC_VLA3'] * u.deg, frame = 'fk5')
    
    seps = list(abs((cosmos_coord.separation(mpa_coords)).to(u.arcsec) / u.arcsec))
    
    min_ind = seps.index(min(seps))
    
    return {'VLA_ID' : tab[min_ind]['ID_VLA3'], 'RA' : tab[min_ind]['RA_VLA3'], 'Dec' : tab[min_ind]['DEC_VLA3']}

In [62]:
def matching_vla_ids(ra, dec, vla_cat):
    arcsec_sep_deg = 30 / (60 * 60)
    
    matching_tab = vla_cat[(vla_cat['RA_VLA3'] > ra - arcsec_sep_deg) & (vla_cat['RA_VLA3'] < ra + arcsec_sep_deg) & (vla_cat['DEC_VLA3'] > dec - arcsec_sep_deg) & (vla_cat['DEC_VLA3'] < dec + arcsec_sep_deg)]
    if len(matching_tab) < 0.5:
        return {'VLA_ID' : np.nan, 'RA' : np.nan, 'Dec' : np.nan}
    elif len(matching_tab) == 1:
        return {'VLA_ID' : matching_tab['ID_VLA3'][0], 'RA' : matching_tab['RA_VLA3'][0], 'Dec' : matching_tab['DEC_VLA3'][0]}
    elif len(matching_tab) > 1:
        return get_correct_vla_id(ra, dec, matching_tab)
    else:
        print('Somethings gone very wrong!!')
        sys.exit()

In [63]:
vla_matched_dict = {}

In [64]:
done_ids = list(vla_matched_dict.keys())
for i in tqdm(list(coords_dict.keys())):
    if i in done_ids:
        continue
    
    vla_matched_dict[i] = matching_vla_ids(coords_dict[i]['RA'], coords_dict[i]['DEC'], vla_cat)

100%|██████████| 3772/3772 [00:48<00:00, 78.36it/s] 


In [65]:
vla_matched_df = (
    pd.DataFrame.from_dict(vla_matched_dict, orient = 'index')
    .dropna()
    .reset_index()
    .rename(columns = {'index':'ID', 0 : 'VLA_ID_arr'})
)

In [66]:
vla_matched_df

,ID,VLA_ID,RA,Dec
0,1304538,10560.0,149.727071,2.652998
1,1310001,4870.0,149.692975,2.663856
2,1308927,4870.0,149.692975,2.663856
3,1311422,4870.0,149.692975,2.663856
4,1324485,3284.0,149.693568,2.674183
...,...,...,...,...
2824,715824,7887.0,150.224451,2.104840
2825,1157587,10945.0,150.425829,2.513874
2826,1035987,7514.0,149.687147,2.394481
2827,1203157,8349.0,150.200830,2.566205


In [67]:
mpa_ids = list(clsf_df.ID)

In [68]:
vla_checked_df = (
    vla_matched_df
    .assign(in_mpa = vla_matched_df.ID.apply(lambda x: 'y' if x in mpa_ids or x in smol_cat_ids else 'n'))
)

In [69]:
len(vla_checked_df)

2829

In [70]:
vla_mpa_checked = vla_checked_df.query('in_mpa == "n"').drop(columns = 'in_mpa')

In [71]:
def dedup_vla(df):
    
    ra = df.RA.iloc[0]
    dec = df.Dec.iloc[0]
    
    if len(df) == 1:
        return df.ID.iloc[0]
    
    coords = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    df_coords = df.assign(coords = df.apply(lambda row: SkyCoord(ra = row.ALPHA_J2000 * u.deg, dec = row.DELTA_J2000 * u.deg, frame = 'fk5'), axis = 1))
    
    df_seps = df_coords.assign(seps = df_coords.coords.apply(lambda x: np.float64(abs(((x.separation(coords)).to(u.arcsec) / u.arcsec)))))
    min_sep = df_seps.seps.min()
    min_row = df_seps.query('seps == @min_sep')
    return int(min_row.ID.iloc[0])

In [72]:
df_vla_merg = (
    vla_mpa_checked
    .merge(df_cat, on = 'ID', how = 'left')
    [['ID', 'VLA_ID', 'RA', 'Dec', 'ALPHA_J2000', 'DELTA_J2000']]
)

In [73]:
vla_dedup_df = pd.DataFrame(df_vla_merg.groupby('VLA_ID').apply(lambda df: dedup_vla(df))).reset_index().rename(columns = {0 : 'ID'})

In [74]:
vla_dict = vla_dedup_df.set_index('ID').to_dict(orient = 'index')

In [75]:
vla_cat_dict = {}

In [76]:
def getting_vla_info(vla_id, vla_cat):
    row = vla_cat[vla_cat['ID_VLA3'] == vla_id]
    ans = row['CLASS'][0]
    
    if len(ans) < 0.5:
        return np.nan
    else:
        return ans

In [77]:
done_ids = list(vla_cat_dict.keys())
for i in tqdm(list(vla_dict.keys())):
    if i in done_ids:
        continue
    
    vla_cat_dict[i] = getting_vla_info(vla_dict[i]['VLA_ID'], vla_cat)

100%|██████████| 685/685 [00:07<00:00, 91.26it/s] 


In [78]:
vla_cat_df = pd.DataFrame.from_dict(vla_cat_dict, orient = 'index').rename(columns = {'index' : 'ID', 0 : 'clsf'}).dropna()
vla_cat_df

,clsf
1217227,HLAGN
191023,HLAGN
1138010,HLAGN
408560,MLAGN
543013,HLAGN
...,...
1181824,HLAGN
419234,HLAGN
518112,HLAGN
1445098,MLAGN


In [79]:
vla_cat_df.clsf.value_counts()

HLAGN    141
MLAGN     94
Name: clsf, dtype: int64

### The Chandra COSMOS Legacy Array

In [80]:
with fits.open(f'{cos_dat_fold}/chandra_COSMOS_legacy_opt_NIR_counterparts_20160113_4d.fits') as hdul:
    chandra_rec = hdul[1].data

In [81]:
# Function that selects the closest object in the MPA_JHU as the galaxy of interest.
def get_correct_chandra_id(ra, dec, tab):
    cosmos_coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    mpa_coords = SkyCoord(ra = tab['RA_x'] * u.deg, dec = tab['DEC_x'] * u.deg, frame = 'fk5')
    
    seps = list(abs((cosmos_coord.separation(mpa_coords)).to(u.arcsec) / u.arcsec))
    
    min_ind = seps.index(min(seps))
    
    return {'chand_ID' : tab[min_ind]['id_x'], 'RA' : tab[min_ind]['RA_x'], 'Dec' : tab[min_ind]['DEC_x']}

In [82]:
def matching_chandra_ids(ra, dec, chandra_rec):
    arcsec_sep_deg = 30 / (60 * 60)
    
    matching_tab = chandra_rec[(chandra_rec['RA_x'] > ra - arcsec_sep_deg) & (chandra_rec['RA_x'] < ra + arcsec_sep_deg) & (chandra_rec['DEC_x'] > dec - arcsec_sep_deg) & (chandra_rec['DEC_x'] < dec + arcsec_sep_deg)]
    if len(matching_tab) < 0.5:
        return {'chand_ID' : np.nan, 'RA' : np.nan, 'Dec' : np.nan}
    elif len(matching_tab) == 1:
        return {'chand_ID' : matching_tab['id_x'][0], 'RA' : matching_tab['RA_x'][0], 'Dec' : matching_tab['DEC_x'][0]}
    elif len(matching_tab) > 1:
        return get_correct_chandra_id(ra, dec, matching_tab)
    else:
        print('Somethings gone very wrong!!')
        sys.exit()

In [83]:
chandra_matched_dict = {}

In [84]:
done_ids = list(chandra_matched_dict.keys())
for i in tqdm(list(coords_dict.keys())):
    if i in done_ids:
        continue
    
    chandra_matched_dict[i] = matching_chandra_ids(coords_dict[i]['RA'], coords_dict[i]['DEC'], chandra_rec)

100%|██████████| 3772/3772 [01:39<00:00, 37.89it/s]


In [85]:
chandra_matched_df = pd.DataFrame.from_dict(chandra_matched_dict, orient = 'index').dropna().reset_index().rename(columns = {'index' : 'ID', 0 : 'chand_ID'})

In [86]:
vla_ids = list(vla_cat_df.reset_index()['index'])

In [87]:
chandra_known = (
    chandra_matched_df
    .assign(in_prior = chandra_matched_df.ID.apply(lambda x: 'y' if x in mpa_ids or x in vla_ids or x in smol_cat_ids else 'n'))
)

In [88]:
chandra_rem = chandra_known.query('in_prior == "n"').drop(columns = 'in_prior')

In [89]:
def dedup_chandra(df):
    
    ra = df.RA.iloc[0]
    dec = df.Dec.iloc[0]
    
    if len(df) == 1:
        return df.ID.iloc[0]
    
    coords = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    df_coords = df.assign(coords = df.apply(lambda row: SkyCoord(ra = row.ALPHA_J2000 * u.deg, dec = row.DELTA_J2000 * u.deg, frame = 'fk5'), axis = 1))
    
    df_seps = df_coords.assign(seps = df_coords.coords.apply(lambda x: np.float64(abs(((x.separation(coords)).to(u.arcsec) / u.arcsec)))))
    min_sep = df_seps.seps.min()
    min_row = df_seps.query('seps == @min_sep')
    return int(min_row.ID.iloc[0])

In [90]:
df_chand_merg = (
    chandra_rem
    .merge(df_cat, on = 'ID', how = 'left')
    [['ID', 'chand_ID', 'RA', 'Dec', 'ALPHA_J2000', 'DELTA_J2000']]
)

In [91]:
chand_dedup_df = pd.DataFrame(df_chand_merg.groupby('chand_ID').apply(lambda df: dedup_chandra(df))).reset_index().rename(columns = {0 : 'ID'})

In [92]:
chand_dedup_df

,chand_ID,ID
0,cid_1010,541164
1,cid_1020,484461
2,cid_1029,473415
3,cid_1039,352373
4,cid_104,1056432
...,...,...
445,lid_721,1020296
446,lid_759,938996
447,lid_969,316890
448,lid_973,327548


In [93]:
def getting_chandra_info(chandra_id, chandra_cat):
    row = chandra_cat[chandra_cat['id_x'] == chandra_id]
    ans = row['spec_type'][0]
    
    
    return ans

In [94]:
chandra_clsf_dict = {}

In [95]:
chandra_dedup_dict = chand_dedup_df.set_index('ID').to_dict(orient = 'index')

In [96]:
done_ids = list(chandra_clsf_dict.keys())
for i in tqdm(list(chandra_dedup_dict.keys())):
    if i in done_ids:
        continue
    
    chandra_clsf_dict[i] = getting_chandra_info(chandra_dedup_dict[i]['chand_ID'], chandra_rec)

100%|██████████| 450/450 [00:09<00:00, 47.12it/s]


In [97]:
chandra_clsf_df = pd.DataFrame.from_dict(chandra_clsf_dict, orient = 'index').reset_index().rename(columns = {'index' : 'ID', 0 : 'clsf'})

In [98]:
chandra_clsf_df.clsf.value_counts()

-9     176
 2     137
 1      84
-99     33
-1      13
 0       6
-2       1
Name: clsf, dtype: int64

In [99]:
def classifying_chandra(clsf):
    if clsf == 1:
        return 'BLAGN'
    elif clsf == 2:
        return 'non-BLAGN'
    else:
        return np.nan

In [100]:
chandra_agn_cat = (
    chandra_clsf_df
    .assign(agn_clsf = chandra_clsf_df.clsf.apply(lambda x: classifying_chandra(x)))
    .drop(columns = ['clsf'])
    .dropna()
)

In [101]:
chandra_agn_cat

,ID,agn_clsf
0,541164,non-BLAGN
2,473415,non-BLAGN
4,1056432,BLAGN
5,1160603,non-BLAGN
8,772549,non-BLAGN
...,...,...
442,1391374,BLAGN
443,1470183,BLAGN
444,1392243,BLAGN
445,1020296,BLAGN


### Putting it all Together!

In [102]:
chandra_agn_cat.head()

,ID,agn_clsf
0,541164,non-BLAGN
2,473415,non-BLAGN
4,1056432,BLAGN
5,1160603,non-BLAGN
8,772549,non-BLAGN


chandra_agn_cat

vla_cat_df

clsf_df

smol_clsf_df

In [103]:
vla_cat_comb = vla_cat_df.reset_index().rename(columns = {'index' : 'ID', 'clsf' : 'agn_clsf' })

In [104]:
vla_cat_comb.drop_duplicates(subset='ID', keep='first')

,ID,agn_clsf
0,1217227,HLAGN
1,191023,HLAGN
2,1138010,HLAGN
3,408560,MLAGN
4,543013,HLAGN
...,...,...
230,1181824,HLAGN
231,419234,HLAGN
232,518112,HLAGN
233,1445098,MLAGN


In [105]:
df_agn_cat = pd.concat([clsf_df, vla_cat_comb, chandra_agn_cat, smol_clsf_df])

In [106]:
len(df_agn_cat)

2564

In [107]:
len(df_agn_cat.drop_duplicates(subset='ID', keep='first'))

2527

In [108]:
df_agn_cat.agn_clsf.value_counts()

SFG              1339
AGN               726
HLAGN             141
non-BLAGN         137
MLAGN              94
BLAGN              84
SF                 29
Composite           7
AGN_Non_Liner       4
AGN_Liner           3
Name: agn_clsf, dtype: int64

In [109]:
df_agn_cat.to_csv(f'{combine_fold}/source-agn-cat.csv')